# Resultados con Print

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options 

# Ruta al ejecutable de ChromeDriver
chrome_driver_path = r'C:\Users\Adri\Downloads\chromedriver-win64\chromedriver.exe'

# Configurar el servicio del controlador de Chrome
chrome_service = ChromeService(executable_path=chrome_driver_path)

# Configurar las opciones de Chrome en modo headless
chrome_options = Options()
chrome_options.add_argument('--headless')  # Habilitar el modo headless

# Iniciar el navegador Chrome con las opciones de configuración
driver = webdriver.Chrome(service=chrome_service, options=chrome_options) 

# URLs de las páginas que deseas scrapear
pages = ['https://www.jdsports.es/hombre/ropa-de-hombre/latest/',
        'https://www.jdsports.es/hombre/calzado-de-hombre/zapatillas/'
        ]

for page in pages:
    soup = BeautifulSoup(requests.get(page).content, 'html.parser')  # obtener el contenido de la página
    product_items = soup.find_all(class_='productListItem')  # obtener todos los productos mediante la etiqueta html de productos

    for product in product_items:
        product_title = product.find(class_='itemTitle').find('a').text  # obtener el título de los productos
        product_price = product.find(class_='itemPrice').find(class_='pri').text  # obtener el precio de los productos
        product_link = "https://www.jdsports.es" + product.find(class_='itemTitle').find('a')['href']  # obtener el enlace de los productos

        # Busca y muestra los colores disponibles
        colors = []  # Lista de colores
        try:
            productSoup = BeautifulSoup(requests.get(product_link).content, 'html.parser')  # obtener el contenido de la página
            color_list = productSoup.find('ul', class_='smScroll').find_all('li')
            for color in color_list:
                img = color.find('img')
                if img:
                    color_name = img['title']
                    colors.append(color_name)
        except Exception as e:
            colors.append('Nan')  # Si no hay colores, agrega 'Nan' a la lista

        driver.get(product_link)  # Abre la página de productos con Selenium

        print(f"º {product_title} | {product_price} | {product_link}")

        size_buttons = driver.find_elements(By.CSS_SELECTOR, 'button[data-e2e="pdp-productDetails-size"]')
        for size_button in size_buttons:
            size = size_button.get_attribute('data-size')
            price = size_button.get_attribute('data-price')
            stock = size_button.get_attribute('data-stock')

            # Si hay colores, formatea la cadena de colores
            if colors:
                if len(colors) > 1:
                    color_string = ', '.join(colors[:-1]) + colors[-1] + '.'
                else:
                    color_string = colors[0] + '.'

            print(f'Talla: {size} | Precio: {price} | Stock: {stock} | Colores: {color_string}')
            
driver.quit()  # Cierra el navegador al final


In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import csv
from datetime import datetime

# Función para mostrar un mensaje de carga
def show_loading_message():
    print("Cargando", end="")
    for _ in range(3):
        for _ in range(3):
            print(".", end="", flush=True)
            time.sleep(1)
        print(" ", end="", flush=True)

# Archivo CSV para escribir los resultados
current_date = datetime.now().strftime("%Y%m%d")
current_date_for_name = datetime.now().strftime("%Y%m%d%H%M%S")
csv_file = open('productosJD' + current_date_for_name + ".csv", 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Título', 'Enlace', 'Colores', 'Precio', 'Moneda', 'Fecha'])

# URLs de las páginas para scrapear
pages = ['https://www.jdsports.es/hombre/ropa-de-hombre/latest/',
         'https://www.jdsports.es/hombre/calzado-de-hombre/zapatillas/'
        ]
nurls = 0
for page in pages:
    show_loading_message()  # Muestra el mensaje de carga
    soup = BeautifulSoup(requests.get(page).content, 'html.parser')  # obtener el contenido de la página
    product_items = soup.find_all(class_='productListItem')  # obtener todos los productos mediante la etiqueta html de productos

    for product in product_items:
        product_title = product.find(class_='itemTitle').find('a').text  # obtener el título de los productos
        product_price = product.find(class_='itemPrice').find(class_='pri').text  # obtener el precio de los productos
        product_price = product_price.replace("€", "")
        product_price = product_price.replace(",", ".")
        product_link = "https://www.jdsports.es" + product.find(class_='itemTitle').find('a')['href']  # obtener el enlace de los productos

        # Busca y muestra los colores disponibles
        colors = []
        try:
            productSoup = BeautifulSoup(requests.get(product_link).content, 'html.parser')  # obtener el contenido de la página
            color_list = productSoup.find('ul', class_='smScroll').find_all('li')
            for color in color_list:
                img = color.find('img')
                if img:
                    color_name = img['title']
                    colors.append(color_name)
        except Exception as e:
            colors.append('Nan')  # Si no hay colores, agrega 'Nan' a la lista

        # Si hay colores, formatea la cadena de colores
        if colors:
            if len(colors) > 1:
                color_string = ', '.join(colors[:-1]) + colors[-1]
            else:
                color_string = colors[0]

        # Escribe los resultados en el archivo CSV
        csv_writer.writerow([product_title, product_link, color_string, product_price, "€", current_date])
        nurls += 1
    print("Listo")  # Indica que se ha completado la carga de la página

csv_file.close()


# Resultados en CSV

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import csv
from datetime import datetime

# Configuración de ChromeDriver para Selenium
chrome_driver_path = r'C:\Users\Adri\Downloads\chromedriver-win64\chromedriver.exe'
chrome_service = ChromeService(executable_path=chrome_driver_path)
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# Archivo CSV para escribir los resultados
current_date = datetime.now().strftime("%Y%m%d")
current_date_for_name = datetime.now().strftime("%Y%m%d%H%M%S")
csv_file = open('productosJD' + current_date_for_name +".csv", 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Título', 'Enlace', 'Talla', 'Stock', 'Colores', 'Precio', 'Moneda', 'Fecha'])

# URLs de las páginas para scrapear
pages = ['https://www.jdsports.es/hombre/ropa-de-hombre/latest/',
         'https://www.jdsports.es/hombre/calzado-de-hombre/zapatillas/'
        ]

for page in pages:
    soup = BeautifulSoup(requests.get(page).content, 'html.parser') # obtener el contenido de la página
    product_items = soup.find_all(class_='productListItem') # obtener todos los productos mediante la etiqueta html de productos

    for product in product_items:
        product_title = product.find(class_='itemTitle').find('a').text # obtener el título de los productos
        product_price = product.find(class_='itemPrice').find(class_='pri').text # obtener el precio de los productos
        product_price = product_price.replace("€", "")
        product_price = product_price.replace(",", ".")
        product_link = "https://www.jdsports.es" + product.find(class_='itemTitle').find('a')['href']  # obtener el enlace de los productos

        # Busca y muestra los colores disponibles
        colors = []
        try:
            productSoup = BeautifulSoup(requests.get(product_link).content, 'html.parser') # obtener el contenido de la página
            color_list = productSoup.find('ul', class_='smScroll').find_all('li')
            for color in color_list:
                img = color.find('img')
                if img:
                    color_name = img['title']
                    colors.append(color_name)
        except Exception as e:
            colors.append('Nan') # Si no hay colores, agrega 'Nan' a la lista

        driver.get(product_link) # Abre la página de productos con Selenium

        size_buttons = driver.find_elements(By.CSS_SELECTOR, 'button[data-e2e="pdp-productDetails-size"]')
        for size_button in size_buttons:
            size = size_button.get_attribute('data-size')
            price = size_button.get_attribute('data-price')
            stock = size_button.get_attribute('data-stock')

            # Si hay colores, formatea la cadena de colores
            if colors:
                if len(colors) > 1:
                    color_string = ', '.join(colors[:-1]) + colors[-1]
                else:
                    color_string = colors[0]

            # Escribe los resultados en el archivo CSV
            csv_writer.writerow([product_title, product_link, size, stock, color_string, product_price, "€", current_date])

csv_file.close()
driver.quit()

# Con el uso de Sitemap

In [ ]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import csv
from datetime import datetime

# Configuración de ChromeDriver para Selenium
chrome_driver_path = r'C:\Users\Adri\Downloads\chromedriver-win64\chromedriver.exe'
chrome_service = ChromeService(executable_path=chrome_driver_path)
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# Crear CSV para exportar los resultados
csv_file = open('productosJD.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
csv_writer.writerow(['Título', 'Precio', 'Enlace', 'Talla', 'Precio Talla', 'Stock', 'Colores', 'Fecha'])

# URL del sitemap
sitemap_url = 'https://www.jdsports.es/sitemaps/es-es_desktop_categories_0.xml'

# Obtener URLs del sitemap
sitemap_response = requests.get(sitemap_url)
sitemap_tree = ET.fromstring(sitemap_response.content)

# Extraer URLs de las entradas del sitemap
urls = [loc.text for loc in sitemap_tree.findall('.//{http://www.sitemaps.org/schemas/sitemap/0.9}loc')]

for page in urls:
    soup = BeautifulSoup(requests.get(page).content, 'html.parser')  # obtener el contenido de la página
    product_items = soup.find_all(class_='productListItem')  # obtener todos los productos mediante la etiqueta html de productos

    for product in product_items:
        product_title = product.find(class_='itemTitle').find('a').text  # obtener el título de los productos
        product_price = product.find(class_='itemPrice').find(class_='pri').text  # obtener el precio de los productos
        product_link = "https://www.jdsports.es" + product.find(class_='itemTitle').find('a')['href']  # obtener el enlace de los productos

        # Busca y muestra los colores disponibles
        colors = []
        try:
            productSoup = BeautifulSoup(requests.get(product_link).content, 'html.parser')  # obtener el contenido de la página
            color_list = productSoup.find('ul', class_='smScroll').find_all('li')
            for color in color_list:
                img = color.find('img')
                if img:
                    color_name = img['title']
                    colors.append(color_name)
        except Exception as e:
            colors.append('Nan')  # Si no hay colores, agrega 'Nan' a la lista

        driver.get(product_link)  # Abre la página de productos con Selenium

        size_buttons = driver.find_elements(By.CSS_SELECTOR, 'button[data-e2e="pdp-productDetails-size"]')
        for size_button in size_buttons:
            size = size_button.get_attribute('data-size')
            price = size_button.get_attribute('data-price')
            stock = size_button.get_attribute('data-stock')

            # Si hay colores, formatea la cadena de colores
            if colors:
                if len(colors) > 1:
                    color_string = ', '.join(colors[:-1]) + colors[-1] + '.'
                else:
                    color_string = colors[0] + '.'

            # Escribe los resultados en el archivo CSV
            csv_writer.writerow([product_title, product_price, product_link, size, price, stock, color_string, current_date])

csv_file.close()
driver.quit()
